<a href="https://colab.research.google.com/github/tungrix/recommender-system/blob/Joel/SDSC3002_recommender_system.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Recommending movies: retrieval with distribution strategy

In this tutorial, we're going to train the same retrieval model as we did in the [basic retrieval](basic_retrieval) tutorial, but with distribution strategy.

We're going to:

1. Get our data and split it into a training and test set.
2. Set up two virtual GPUs and TensorFlow MirroredStrategy.
3. Implement a retrieval model using MirroredStrategy.
4. Fit it with MirrorredStrategy and evaluate it.



## Imports


Let's first get our imports out of the way.

In [ ]:
!pip install -q tensorflow-recommenders
!pip install -q --upgrade tensorflow-datasets

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 96.2/96.2 kB 4.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.4/5.4 MB 49.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.0/3.0 MB 101.3 MB/s eta 0:00:00


In [ ]:
import os
import pprint
import tempfile

from typing import Dict, Text

import numpy as np
import tensorflow as tf
import tensorflow_datasets as tfds

In [ ]:
import tensorflow_recommenders as tfrs

## Preparing the dataset

We prepare the dataset in exactly the same way as we do in the [basic retrieval](basic_retrieval) tutorial.

In [ ]:
# Ratings data.
ratings = tfds.load("movielens/100k-ratings", split="train")
# Features of all the available movies.
movies = tfds.load("movielens/100k-movies", split="train")

for x in ratings.take(1).as_numpy_iterator():
  pprint.pprint(x)

for x in movies.take(1).as_numpy_iterator():
  pprint.pprint(x)

Dl Completed...: 0 url [00:00, ? url/s]

Dl Size...: 0 MiB [00:00, ? MiB/s]

Extraction completed...: 0 file [00:00, ? file/s]

Generating splits...:   0%|          | 0/1 [00:00<?, ? splits/s]

Generating train examples...:   0%|          | 0/100000 [00:00<?, ? examples/s]

...

Extraction completed...: 0 file [00:00, ? file/s]

Generating splits...:   0%|          | 0/1 [00:00<?, ? splits/s]

Generating train examples...:   0%|          | 0/1682 [00:00<?, ? examples/s]

Shuffling /root/tensorflow_datasets/movielens/100k-movies/0.1.1.incompleteK7SBTZ/movielens-train.tfrecord*...:…

Dataset movielens downloaded and prepared to /root/tensorflow_datasets/movielens/100k-movies/0.1.1. Subsequent calls will reuse this data.
{'bucketized_user_age': 45.0,
 'movie_genres': array([7]),
 'movie_id': b'357',
 'movie_title': b"One Flew Over the Cuckoo's Nest (1975)",
 'raw_user_age': 46.0,
 'timestamp': 879024327,
 'user_gender': True,
 'user_id': b'138',
 'user_occupation_label': 4,
 'user_occupation_text': b'doctor',
 'user_rating': 4.0,
 'user_zip_code': b'53211'}
{'movie_genres': array([4]),
 'movie_id': b'1681',
 'movie_title': b'You So Crazy (1994)'}


In [ ]:
for x in ratings.take(1).as_numpy_iterator():
  pprint.pprint(x["movie_genres"])

array([7])


Given the overlapping nature of some of the features in the dataset (see `'bucketized_user_age'`/`'raw_user_age'`, `'user_occupation_label'`/`'user_occupation_text'`)

In [ ]:
ratings = ratings.map(lambda x: {
    "movie_title": x["movie_title"],
    # "movie_genres": x["movie_genres"],
    "user_id": x["user_id"],
    "user_rating": x["user_rating"],
    "user_occupation_label": x["user_occupation_label"],
    "bucketized_user_age": x["bucketized_user_age"],
    "user_gender": int(x["user_gender"])
})
movies = movies.map(lambda x: {
    "movie_title": x["movie_title"],
    # "movie_genres": x["movie_genres"]
    })

tf.random.set_seed(42)
shuffled = ratings.shuffle(100_000, seed=42, reshuffle_each_iteration=False)

train = shuffled.take(80_000)
test = shuffled.skip(80_000).take(20_000)

movie_titles = movies.batch(1_000).map(lambda x: x["movie_title"])
user_ids = ratings.batch(1_000_000).map(lambda x: x["user_id"])

In [ ]:
user_ratings = ratings.batch(1_000_000).map(lambda x: x["user_rating"])
unique_ratings = np.unique(np.concatenate(list(user_ratings)))

In [ ]:
user_occupations = ratings.batch(1_000_000).map(lambda x: x["user_occupation_label"])
unique_occupations = np.unique(np.concatenate(list(user_occupations)))

In [ ]:
user_ages = ratings.batch(1_000_000).map(lambda x: x["bucketized_user_age"])
unique_ages = np.unique(np.concatenate(list(user_ages)))

In [ ]:
user_genders = ratings.batch(1_000_000).map(lambda x: x["user_gender"])
unique_genders = np.unique(np.concatenate(list(user_genders)))

In [ ]:
unique_movie_titles = np.unique(np.concatenate(list(movie_titles)))
unique_user_ids = np.unique(np.concatenate(list(user_ids)))

unique_movie_titles[:10]

array([b"'Til There Was You (1997)", b'1-900 (1994)',
       b'101 Dalmatians (1996)', b'12 Angry Men (1957)', b'187 (1997)',
       b'2 Days in the Valley (1996)',
       b'20,000 Leagues Under the Sea (1954)',
       b'2001: A Space Odyssey (1968)',
       b'3 Ninjas: High Noon At Mega Mountain (1998)',
       b'39 Steps, The (1935)'], dtype=object)

## Set up two virtual GPUs

If you have not added GPU accelerators to your Colab, please disconnect the Colab runtime and do it now. We need the GPU to run the code below:

In [ ]:
gpus = tf.config.list_physical_devices("GPU")
if gpus:
  # Create 2 virtual GPUs with 1GB memory each
  try:
    tf.config.set_logical_device_configuration(
        gpus[0],
        [tf.config.LogicalDeviceConfiguration(memory_limit=1024),
         tf.config.LogicalDeviceConfiguration(memory_limit=1024)])
    logical_gpus = tf.config.list_logical_devices("GPU")
    print(len(gpus), "Physical GPU,", len(logical_gpus), "Logical GPUs")
  except RuntimeError as e:
    # Virtual devices must be set before GPUs have been initialized
    print(e)

strategy = tf.distribute.MirroredStrategy()

Virtual devices cannot be modified after being initialized


## Implementing a model

We implement the user_model, movie_model, metrics and task in the same way as we do in the [basic retrieval](basic_retrieval) tutorial, but we wrap them in the distribution strategy scope:

In [ ]:
embedding_dimension = 32

In [ ]:
class UserModel(tf.keras.Model):

  def __init__(self):
    super().__init__()

    self.user_embedding = tf.keras.Sequential([
        tf.keras.layers.StringLookup(
          vocabulary=unique_user_ids, mask_token=None),
        # We add an additional embedding to account for unknown tokens.
        tf.keras.layers.Embedding(len(unique_user_ids) + 1, embedding_dimension)
    ])
    # self.timestamp_embedding = tf.keras.Sequential([
    #   tf.keras.layers.Discretization(timestamp_buckets.tolist()),
    #   tf.keras.layers.Embedding(len(timestamp_buckets) + 2, 32)
    # ])
    # self.normalized_timestamp = tf.keras.layers.Normalization(
    #     axis=None
    # )
    self.user_age = tf.keras.layers.experimental.preprocessing.Normalization()

    self.user_occupation = tf.keras.layers.experimental.preprocessing.Normalization()

    self.user_gender = tf.keras.layers.experimental.preprocessing.Normalization()

  def call(self, inputs):

    # Take the input dictionary, pass it through each input layer,
    # and concatenate the result.
    return tf.concat([
        self.user_embedding(inputs["user_id"]),
        self.user_age(inputs["bucketized_user_age"]),
        self.user_occupation(inputs["user_occupation_label"]),
        self.user_gender(inputs["user_gender"])

        # self.timestamp_embedding(inputs["timestamp"]),
        # tf.reshape(self.normalized_timestamp(inputs["timestamp"]), (-1, 1))
    ], axis=1)

In [ ]:
class MovieModel(tf.keras.Model):

  def __init__(self):
    super().__init__()

    max_tokens = 10_000

    self.title_embedding = tf.keras.Sequential([
      tf.keras.layers.StringLookup(
          vocabulary=unique_movie_titles, mask_token=None),
      tf.keras.layers.Embedding(len(unique_movie_titles) + 1, embedding_dimension)
    ])
    self.title_text_embedding = tf.keras.Sequential([
      tf.keras.layers.TextVectorization(max_tokens=max_tokens),
      tf.keras.layers.Embedding(max_tokens, embedding_dimension, mask_zero=True),
      # We average the embedding of individual words to get one embedding vector
      # per title.
      tf.keras.layers.GlobalAveragePooling1D(),
    ])

  def call(self, inputs):
    return tf.concat([
        self.title_embedding(inputs["movie_title"]),
        self.title_text_embedding(inputs["movie_title"]),
    ], axis=1)


In [ ]:
# movie_model = MovieModel()

# movie_model.title_text_embedding.layers[0].adapt(
#     ratings.map(lambda x: x["movie_title"]))

# for row in ratings.batch(1).take(1):
#   print(f"Computed representations: {movie_model(row)[0, :3]}")

In [ ]:
embedding_dimension = 32

with strategy.scope():
  user_model = UserModel()

  movie_model = MovieModel()

  metrics = tfrs.metrics.FactorizedTopK(
    candidates=movies.batch(128).map(movie_model)
  )

  task = tfrs.tasks.Retrieval(
    metrics=metrics
  )

We can now put it all together into a model. This is exactly the same as in the [basic retrieval](basic_retrieval) tutorial.

In [ ]:
class MovielensModel(tfrs.Model):

  def __init__(self, user_model, movie_model):
    super().__init__()
    self.movie_model = movie_model
    self.user_model = user_model
    self.task = task

  def compute_loss(self, features: Dict[Text, tf.Tensor], training=False) -> tf.Tensor:
    # We pick out the user features and pass them into the user model.
    user_embeddings = self.user_model([features["user_id"], features["bucketized_user_age"], features["user_occupation_label"], features['user_gender']])
    # And pick out the movie features and pass them into the movie model,
    # getting embeddings back.
    positive_movie_embeddings = self.movie_model(features["movie_title"])

    # The task computes the loss and the metrics.
    return self.task(user_embeddings, positive_movie_embeddings)

## Fitting and evaluating

Now we instantiate and compile the model within the distribution strategy scope.

Note that we are using Adam optimizer here instead of Adagrad as in the [basic retrieval](basic_ranking) tutorial since Adagrad is not supported here.

In [ ]:
with strategy.scope():
  model = MovielensModel(user_model, movie_model)
  model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.1))

Then shuffle, batch, and cache the training and evaluation data.

In [ ]:
cached_train = train.shuffle(100_000).batch(8192).cache()
cached_test = test.batch(4096).cache()

Then train the  model:

In [ ]:
model.fit(cached_train, epochs=3)

Epoch 1/3


KeyError: ignored

You can see from the training log that TFRS is making use of both virtual GPUs.

Finally, we can evaluate our model on the test set:

In [ ]:
model.evaluate(cached_test, return_dict=True)

5/5 [==============================] - 7s 754ms/step - factorized_top_k/top_1_categorical_accuracy: 1.0000e-04 - factorized_top_k/top_5_categorical_accuracy: 0.0015 - factorized_top_k/top_10_categorical_accuracy: 0.0059 - factorized_top_k/top_50_categorical_accuracy: 0.0653 - factorized_top_k/top_100_categorical_accuracy: 0.1566 - loss: 32349.5046 - regularization_loss: 0.0000e+00 - total_loss: 32349.5046


{'factorized_top_k/top_1_categorical_accuracy': 9.999999747378752e-05,
 'factorized_top_k/top_5_categorical_accuracy': 0.001500000013038516,
 'factorized_top_k/top_10_categorical_accuracy': 0.005900000222027302,
 'factorized_top_k/top_50_categorical_accuracy': 0.06525000184774399,
 'factorized_top_k/top_100_categorical_accuracy': 0.1565999984741211,
 'loss': 29401.66796875,
 'regularization_loss': 0,
 'total_loss': 29401.66796875}

This concludes the retrieval with distribution strategy tutorial.